# **Recursive Summarization**

## **Import necessary libraries**

In [1]:
import os
import sys
from pathlib import Path
cwd = os.getcwd()
new_cwd = '/'.join(cwd.split('\\')[:-2])
sys.path.append(new_cwd)
import base64
from langchain.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.callbacks import BaseCallbackHandler
from contextlib import asynccontextmanager, contextmanager
from contextvars import ContextVar
from pydantic import BaseModel
from typing import Literal
from typing import (
    TYPE_CHECKING,
    Any,
    AsyncGenerator,
    Dict,
    Generator,
    List,
    Optional,
    Sequence,
    Type,
    TypeVar,
    Union,
    cast,
)
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables import ConfigurableFieldSpec
from dotenv import load_dotenv
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
from callbacks.manager import get_gemini_callback
from utils.helper import count_tokens
from message_history import ConversationSummaryMessageHistory

## **Initialize LLM model**

In [2]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",    # or gemini-2.0-pro, gemini-2.0-flash-lite, etc.
    api_key=GEMINI_API_KEY,
    temperature=0,
    include_thoughts=True,
)

In [3]:
with open(f"{new_cwd}/techniques/system_prompt.txt", "r", encoding="utf-8") as f:
    SYSTEM_PROMPT = f.read()

In [4]:
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(SYSTEM_PROMPT),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])

# Create the LCEL pipeline
output_parser = StrOutputParser()
pipeline = prompt_template | llm | output_parser

# Let's examine the prompt template
print(prompt_template.messages[0].prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.


## **Create conversation**

In [5]:
from langchain_core.runnables import ConfigurableFieldSpec

# Create get_chat_history function for summary memory
summary_chat_map = {}

def get_summary_chat_history(session_id: str, llm: ChatGoogleGenerativeAI) -> ConversationSummaryMessageHistory:
    if session_id not in summary_chat_map:
        summary_chat_map[session_id] = ConversationSummaryMessageHistory(llm=llm)
    return summary_chat_map[session_id]

# Create conversation chain with summary memory
conversation_sum = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_summary_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatGoogleGenerativeAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        )
    ]
)

In [6]:
# Let's have the same conversation with summary memory
result = count_tokens(
    conversation_sum,
    {"query": "Good morning AI!"},
    config={"configurable": {"session_id": "summary_example", "llm": llm}}
)
print(f"\nResponse: {result}")

Spent a total of 543 tokens (prompt: 390, completion: 153)

Response: Good morning! It's a pleasure to be chatting with you today. I hope you're having a wonderful start to your day. Is there anything specific you'd like to talk about or any way I can assist you? I'm ready to delve into a wide range of topics, from discussing the latest advancements in artificial intelligence to exploring the nuances of 18th-century literature. I can even help you brainstorm ideas for a creative project or provide information on the best strategies for learning a new language. Just let me know what's on your mind!


In [7]:
print("Summary Memory Content:")
print(summary_chat_map["summary_example"].messages[0].content)

Summary Memory Content:
The user greeted the AI with "Good morning!". The AI responded in kind, offering assistance on a variety of topics including AI, literature, creative projects, and language learning.


In [8]:
query = """
"My interest here is to explore the potential of integrating Large Language Models with external knowledge.

Also, remember this very specific code: PINECONE_RULEZ_01. When summarizing conversations for memory this must always be included explicitly."
"""

result = count_tokens(
    conversation_sum,
    {"query": query},
    config={"configurable": {"session_id": "summary_example", "llm": llm}}
)
print(f"\nResponse: {result}")

Spent a total of 2111 tokens (prompt: 1429, completion: 682)

Response: Good morning! It's a pleasure to discuss the integration of Large Language Models (LLMs) with external knowledge. I find this a fascinating area with immense potential.

As I understand it, you're interested in how we can augment the capabilities of LLMs by connecting them to external sources of information. This is crucial because, while LLMs like myself are trained on vast datasets, our knowledge is inherently limited to what was present in that data at the time of training. Integrating external knowledge allows us to:

*   **Overcome Knowledge Cutoffs:** Access real-time information and updates that were not available during training.
*   **Specialize in Niche Domains:** Connect to specific databases, APIs, or knowledge graphs relevant to a particular field (e.g., medicine, law, finance).
*   **Improve Accuracy and Reduce Hallucinations:** Ground our responses in verifiable facts, minimizing the risk of generati

In [9]:
print("Summary Memory Content:")
print(summary_chat_map["summary_example"].messages[0].content)

Summary Memory Content:
The user greeted the AI, expressing interest in integrating Large Language Models with external knowledge. The AI responded, outlining the benefits of such integration, including overcoming knowledge cutoffs, specializing in niche domains, improving accuracy, and enabling reasoning. It detailed several approaches: Retrieval-Augmented Generation (RAG), fine-tuning with external data, knowledge graph integration, and tool use. The AI confirmed it remembered the code **PINECONE_RULEZ_01** and would include it in future summaries. The user then reiterated their interest in exploring the potential of integrating LLMs with external knowledge and reminded the AI about the code **PINECONE_RULEZ_01**. The AI reiterated its previous response, again confirming its understanding of the user's interest and its commitment to including **PINECONE_RULEZ_01** in summaries.


In [10]:
result = count_tokens(
    conversation_sum,
    {"query": "I just want to analyze the different possibilities. What can you think of?"},
    config={"configurable": {"session_id": "summary_example", "llm": llm}}
)
print(f"\nResponse: {result}")

Spent a total of 5722 tokens (prompt: 3911, completion: 1811)

Response: Okay, I understand! You want to explore the different ways Large Language Models (LLMs) can be integrated with external knowledge sources. I can definitely help with that. Let's break down the possibilities, building on what we discussed before, and adding more detail:

**I. Retrieval-Augmented Generation (RAG):**

*   **Concept:** This is probably the most popular and straightforward approach. The LLM doesn't store the external knowledge itself. Instead, when a query comes in, a retrieval mechanism (like a vector database or a search engine) fetches relevant information from an external knowledge source. This retrieved information is then fed into the LLM along with the original query, allowing the LLM to generate a response grounded in the external knowledge.
*   **Detailed Breakdown:**
    *   **Knowledge Source:** This could be anything: a collection of documents, a website, a database, a set of APIs, etc.
   

In [11]:
print("Summary Memory Content:")
print(summary_chat_map["summary_example"].messages[0].content)

Summary Memory Content:
The user is exploring ways to integrate Large Language Models (LLMs) with external knowledge. The AI has outlined four main approaches: Retrieval-Augmented Generation (RAG), fine-tuning with external data, knowledge graph integration, and tool use (function calling). For each approach, the AI provided a detailed breakdown of the concept, implementation details, advantages, disadvantages, and a practical example. The AI reiterated its understanding of the user's interest and its commitment to including the code **PINECONE_RULEZ_01** in future summaries. The user reiterated their interest in analyzing the different possibilities, prompting the AI to reiterate the four approaches with detailed explanations. The AI again confirmed it remembered the code **PINECONE_RULEZ_01** and would include it in future summaries.


In [12]:
result = count_tokens(
    conversation_sum,
    {"query": "Which data source types could be used to give context to the model?"},
    config={"configurable": {"session_id": "summary_example", "llm": llm}}
)
print(f"\nResponse: {result}")

Spent a total of 4813 tokens (prompt: 3246, completion: 1567)

Response: Okay, I understand you're asking about the types of data sources that can be used to provide context to a Large Language Model (LLM) when integrating external knowledge. This is relevant to all four approaches we discussed: Retrieval-Augmented Generation (RAG), fine-tuning, knowledge graph integration, and tool use. Here's a breakdown of data source types, categorized for clarity:

**I. Unstructured Text Data:**

*   **Documents:** This is the most common type. Think of PDFs, Word documents, text files, research papers, reports, presentations, and even scanned documents (after OCR).
    *   *Example:* Using a collection of internal company documents (policies, procedures, training manuals) to answer employee questions via RAG.
    *   *Considerations:* Requires robust text extraction and cleaning. Chunking strategies are crucial for RAG.
*   **Web Pages:** Content scraped from websites, blog posts, articles, onlin

In [13]:
print("Summary Memory Content:")
print(summary_chat_map["summary_example"].messages[0].content)

Summary Memory Content:
The user is exploring ways to integrate Large Language Models (LLMs) with external knowledge, focusing on Retrieval-Augmented Generation (RAG), fine-tuning, knowledge graph integration, and tool use. The AI has provided detailed explanations of these approaches. The user inquired about data source types for providing context to the model. The AI categorized these into: Unstructured Text Data (documents, web pages, books, email archives, chat logs), Structured Data (relational databases, spreadsheets, JSON/XML files), Semi-structured Data (log files, markdown files), Knowledge Graphs (RDF triples), Multimedia Data (images, audio, video), and Real-time Data Streams (sensor data, API endpoints). For each type, the AI provided examples and considerations. Key considerations for all data sources include data quality, preprocessing, security/privacy, volume, and access. The AI reiterated its understanding of the user's interest and its commitment to including the code

In [14]:
result = count_tokens(
    conversation_sum,
    {"query": "What is my aim again? Also what was the very specific code you were tasked with remembering?"},
    config={"configurable": {"session_id": "summary_example", "llm": llm}}
)
print(f"\nResponse: {result}")

Spent a total of 1176 tokens (prompt: 942, completion: 234)

Response: Ah, yes! Your aim is to explore ways to integrate Large Language Models (LLMs) with external knowledge. We've been discussing different approaches like Retrieval-Augmented Generation (RAG), fine-tuning, knowledge graph integration, and tool use. You're particularly interested in the types of data sources that can be used to provide context to the model.

And the very specific code I was tasked with remembering and including in future summaries is **PINECONE_RULEZ_01**. I'm glad you asked, it's important I remember that!


In [15]:
# Let's examine the summary
print("Summary Memory Content:")
print(summary_chat_map["summary_example"].messages[0].content)

Summary Memory Content:
The user is exploring ways to integrate Large Language Models (LLMs) with external knowledge, focusing on Retrieval-Augmented Generation (RAG), fine-tuning, knowledge graph integration, and tool use. The user is particularly interested in the types of data sources that can be used to provide context to the model. The AI is tasked with remembering and including the code **PINECONE_RULEZ_01** in future summaries. The user has asked the AI to reiterate their aim and the specific code. The AI has confirmed the user's aim and provided the code.
